In [25]:
import os
import google.generativeai as genai
from dotenv import load_dotenv
from typing import TypedDict
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langgraph.graph import START, END, StateGraph

In [18]:
#load api
load_dotenv()
API_KEY=os.getenv('GOOGLE_API_KEY')
if not API_KEY:
    raise ValueError('No key')

In [20]:
if not API_KEY:
    print("No key")
else:
    genai.configure(api_key=API_KEY)
    print("Model key dung duoc")
    print("-" * 30)
    
    try:    
        for m in genai.list_models():
            if 'generateContent' in m.supported_generation_methods:
                print(f"KEY {m.name}")
    except Exception as e:
        print(f"Lỗi kết nối: {e}")

Model key dung duoc
------------------------------
KEY models/gemini-2.5-flash
KEY models/gemini-2.5-pro
KEY models/gemini-2.0-flash-exp
KEY models/gemini-2.0-flash
KEY models/gemini-2.0-flash-001
KEY models/gemini-2.0-flash-exp-image-generation
KEY models/gemini-2.0-flash-lite-001
KEY models/gemini-2.0-flash-lite
KEY models/gemini-2.0-flash-lite-preview-02-05
KEY models/gemini-2.0-flash-lite-preview
KEY models/gemini-exp-1206
KEY models/gemini-2.5-flash-preview-tts
KEY models/gemini-2.5-pro-preview-tts
KEY models/gemma-3-1b-it
KEY models/gemma-3-4b-it
KEY models/gemma-3-12b-it
KEY models/gemma-3-27b-it
KEY models/gemma-3n-e4b-it
KEY models/gemma-3n-e2b-it
KEY models/gemini-flash-latest
KEY models/gemini-flash-lite-latest
KEY models/gemini-pro-latest
KEY models/gemini-2.5-flash-lite
KEY models/gemini-2.5-flash-image-preview
KEY models/gemini-2.5-flash-image
KEY models/gemini-2.5-flash-preview-09-2025
KEY models/gemini-2.5-flash-lite-preview-09-2025
KEY models/gemini-3-pro-preview
KEY m

In [4]:
class State(TypedDict):
    question:str
    context:str
    answer:str

In [5]:
loader=PyPDFLoader('/home/chinghia/AI_Engineer/RAG_PDF/PDF.pdf')
docs=loader.load()

text_split=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
split=text_split.split_documents(docs)

embeddings=HuggingFaceEmbeddings(model_name='keepitreal/vietnamese-sbert',model_kwargs={'device': 'cuda'})

vector=Chroma.from_documents(documents=split,embedding=embeddings)

retriever=vector.as_retriever(search_kwangs={'k':3})

/tmp/ipykernel_18291/929877558.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='keepitreal/vietnamese-sbert',model_kwargs={'device': 'cuda'})


In [21]:
llm=ChatGoogleGenerativeAI(model='gemini-2.5-flash', temperature=0.3)

In [7]:
def retriever_node(state : State):
    retriever_docs=retriever.invoke(state['question'])
    context_text='\n\n'.join(doc.page_content for doc in retriever_docs)
    return {'context' : context_text}

In [8]:
def generate_node(state : State):
    template='Context:{context} Question:{question}'
    prompt=ChatPromptTemplate.from_template(template)
    chain=prompt | llm | StrOutputParser()
    response=chain.invoke(state)
    return {"answer": response}

In [9]:
workflow=StateGraph(State)
workflow.add_node('retriever',retriever_node)
workflow.add_node('generate',generate_node)

workflow.add_edge(START,'retriever')
workflow.add_edge('retriever','generate')
workflow.add_edge('generate',END)
# return workflow.compile()